In [1]:
import numpy as np 
import pandas as pd
import torch
import tiktoken 

In [ ]:
x = np.array([[1,2,3]])
print(x.shape)
x

In [ ]:
y = np.array([[1],[2],[3]])
print(y.shape)
y

In [ ]:
x + y

In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])

# create a 3D tensor from a nested Python list
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [ ]:
tensor2d

In [ ]:
tensor2d.T

In [ ]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

loss.backward()

print(w1.grad)
print(b.grad)

In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model)
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

In [ ]:
model.layers[0].weight

In [ ]:
print(model.layers[0].weight.shape)

In [ ]:
torch.manual_seed(123)

X = torch.rand((1, 50))
with torch.no_grad():
    out = model(X)
print(out)

In [ ]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [ ]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]


train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2,num_outputs=2)

#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # considered pytorch best practice
model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5)
epochs = 3

for epoch in range(epochs):

    model.train()

    for batch_idx, (features,labels) in enumerate(train_loader):
        features,labels = features.to(device), labels.to(device)

        logits = model(features)
        loss = F.cross_entropy(logits,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{epochs:03d}" f" | Batch {batch_idx:03d}/{len(train_loader):03d}" f" | Train/Val Loss: {loss:.2f}")
      
    model.eval()

In [ ]:
# can make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train) # logits
print(outputs)

torch.set_printoptions(sci_mode=False) 
probs = torch.softmax(outputs,dim=1) # class probas
print(probs)

preds = torch.argmax(probs,dim=1) # convert probas to class preds
print(preds)
 
preds = torch.argmax(outputs,dim=1) # could also get preds direct from logits
print(preds)

In [ ]:
(torch.sum(preds == y_train) / len(y_train))*100 # acc

In [ ]:
for key in model.state_dict(): 
    print(key)

In [ ]:
torch.save(model.state_dict(),"model_test.pth")

In [ ]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model_test.pth", weights_only=True))

In [ ]:
#model = MyTransformerDecoderModel(...)
#model = model.half().cuda()  # Use float16 to save VRAM

# Check memory usage
#print(torch.cuda.memory_allocated() / 1024 ** 2, "MB allocated")

In [ ]:
x = torch.randn(2,2,3)
y = torch.randn(2,2,3)
y = y.view()
print(x)
print(x.shape)
print("")
print(y)
print(y.shape)

In [ ]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = enc.n_vocab
print(f"Vocab size: {vocab_size}")
es = enc.encode("Hello, Marcus Aurelius.")
print(es)
ds = enc.decode(es)
print(ds)

### Multi-head tensor

In [3]:
# optimal mha
# no loops, one big matrix 
B = 2 # batch size
T = 2 # sequence length
d = 3 # emebedding dim 
hd = 4 # head dim, in practice it will be d // n_heads
nh = 3 # num heads

#X = torch.randint(low=1,high=9,size=(B,T,d))
X = torch.randn((B,T,d))

In [ ]:
ll = torch.nn.Linear(d,3*hd*nh,bias=True)
QKV = ll(X)
print(QKV)
QKV.shape

In [ ]:
qkv = QKV.view(B,T,nh,3,hd) 
print(qkv.shape)

#qkv = QKV.view(B,nh,3,T,hd)
#print(qkv.shape)

In [ ]:
qkv[:,:,0,1,:] # K of head 1

In [ ]:
qkv[:,:,:,1,:] # All Ks

In [ ]:
# Q @ K.T for all heads
attn_matrix = qkv[:,:,:,0,:] @ qkv[:,:,:,1,:].transpose(-2,-1)
# [2, 2, 3, 4] @ [2, 2, 4, 3] = [2, 2, 3, 3]
# [B, T, nh, hd] @ [B, T, hd, nh] = [B, T, nh, nh] --> the attention matrix for each head of each sequence of each batch
print(attn_matrix)
print(attn_matrix.shape)

In [ ]:
qkv[:,:,0,0,:] # Q of head 1 across batches

In [ ]:
qkv[:,:,1,0,:] # Q of head 2 across batches

In [ ]:
QKV[:,:,0*hd:1*hd] # Q of head 1 across batches
#QKV[:,:,1*hd:2*hd] # K of head 1
#QKV[:,:,2*hd:3*hd] # V of head 1
#QKV[:,:,3*hd:4*hd] # Q of head 2

In [ ]:
# Q @ K.T
QKV[:,:,:hd] @ QKV[:,:,hd:2*hd].transpose(-2,-1)
# [B,T,hd] @ [B,hd,T] --> [B,T,T]

### Playing around with view()

simulate self attention calculation

In [85]:
total = B*T*hd*nh*3
QKV = torch.arange(1,total+1).view(B,T,nh*3*hd)
#QKV = torch.randn(B,T,nh*3*hd)
QKV # output of linear layer [B,T,KQV*nh*hd]

tensor([[[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
           15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
           29,  30,  31,  32,  33,  34,  35,  36],
         [ 37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
           51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
           65,  66,  67,  68,  69,  70,  71,  72]],

        [[ 73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
           87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
          101, 102, 103, 104, 105, 106, 107, 108],
         [109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
          123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
          137, 138, 139, 140, 141, 142, 143, 144]]])

In [86]:
qkv = QKV.view(B,nh,T,3,hd) 
# everything grouped by batch -> head -> sequence -> QKV matrix
qkv

tensor([[[[[  1,   2,   3,   4],
           [  5,   6,   7,   8],
           [  9,  10,  11,  12]],

          [[ 13,  14,  15,  16],
           [ 17,  18,  19,  20],
           [ 21,  22,  23,  24]]],


         [[[ 25,  26,  27,  28],
           [ 29,  30,  31,  32],
           [ 33,  34,  35,  36]],

          [[ 37,  38,  39,  40],
           [ 41,  42,  43,  44],
           [ 45,  46,  47,  48]]],


         [[[ 49,  50,  51,  52],
           [ 53,  54,  55,  56],
           [ 57,  58,  59,  60]],

          [[ 61,  62,  63,  64],
           [ 65,  66,  67,  68],
           [ 69,  70,  71,  72]]]],



        [[[[ 73,  74,  75,  76],
           [ 77,  78,  79,  80],
           [ 81,  82,  83,  84]],

          [[ 85,  86,  87,  88],
           [ 89,  90,  91,  92],
           [ 93,  94,  95,  96]]],


         [[[ 97,  98,  99, 100],
           [101, 102, 103, 104],
           [105, 106, 107, 108]],

          [[109, 110, 111, 112],
           [113, 114, 115, 116],
           [117

In [87]:
Q = qkv[:,:,:,0,:] # all Q matrices 
print(Q)
print(Q.shape)

tensor([[[[  1,   2,   3,   4],
          [ 13,  14,  15,  16]],

         [[ 25,  26,  27,  28],
          [ 37,  38,  39,  40]],

         [[ 49,  50,  51,  52],
          [ 61,  62,  63,  64]]],


        [[[ 73,  74,  75,  76],
          [ 85,  86,  87,  88]],

         [[ 97,  98,  99, 100],
          [109, 110, 111, 112]],

         [[121, 122, 123, 124],
          [133, 134, 135, 136]]]])
torch.Size([2, 3, 2, 4])


In [88]:
K = qkv[:,:,:,1,:] # all K matrices
print(K)
print(K.shape)

tensor([[[[  5,   6,   7,   8],
          [ 17,  18,  19,  20]],

         [[ 29,  30,  31,  32],
          [ 41,  42,  43,  44]],

         [[ 53,  54,  55,  56],
          [ 65,  66,  67,  68]]],


        [[[ 77,  78,  79,  80],
          [ 89,  90,  91,  92]],

         [[101, 102, 103, 104],
          [113, 114, 115, 116]],

         [[125, 126, 127, 128],
          [137, 138, 139, 140]]]])
torch.Size([2, 3, 2, 4])


In [89]:
K_T = qkv[:,:,:,1,:].transpose(-2,-1) # all K matrices transposed
print(K_T)
print(K_T.shape)

tensor([[[[  5,  17],
          [  6,  18],
          [  7,  19],
          [  8,  20]],

         [[ 29,  41],
          [ 30,  42],
          [ 31,  43],
          [ 32,  44]],

         [[ 53,  65],
          [ 54,  66],
          [ 55,  67],
          [ 56,  68]]],


        [[[ 77,  89],
          [ 78,  90],
          [ 79,  91],
          [ 80,  92]],

         [[101, 113],
          [102, 114],
          [103, 115],
          [104, 116]],

         [[125, 137],
          [126, 138],
          [127, 139],
          [128, 140]]]])
torch.Size([2, 3, 4, 2])


In [90]:
V = qkv[:,:,:,2,:]
print(V)
print(V.shape)

tensor([[[[  9,  10,  11,  12],
          [ 21,  22,  23,  24]],

         [[ 33,  34,  35,  36],
          [ 45,  46,  47,  48]],

         [[ 57,  58,  59,  60],
          [ 69,  70,  71,  72]]],


        [[[ 81,  82,  83,  84],
          [ 93,  94,  95,  96]],

         [[105, 106, 107, 108],
          [117, 118, 119, 120]],

         [[129, 130, 131, 132],
          [141, 142, 143, 144]]]])
torch.Size([2, 3, 2, 4])


In [91]:
AM = Q @ K_T
print(AM)
print(AM.shape)
# goal: [B,nh,T,T]

tensor([[[[   70,   190],
          [  382,  1078]],

         [[ 3238,  4510],
          [ 4702,  6550]],

         [[11014, 13438],
          [13630, 16630]]],


        [[[23398, 26974],
          [27166, 31318]],

         [[40390, 45118],
          [45310, 50614]],

         [[61990, 67870],
          [68062, 74518]]]])
torch.Size([2, 3, 2, 2])


In [ ]:
from math import sqrt 

AM = AM / sqrt(d) # scale
mask = torch.triu(torch.ones_like(AM),diagonal=1).bool()
masked_AM = AM.masked_fill(mask,float('-inf'))
attn_weights = torch.nn.functional.softmax(masked_AM,dim=-1)
out = attn_weights @ V
print(attn_weights)
print(attn_weights.shape)

tensor([[[[1.0000, 0.0000],
          [0.4933, 0.5067]],

         [[1.0000, 0.0000],
          [0.5234, 0.4766]],

         [[1.0000, 0.0000],
          [0.4966, 0.5034]]],


        [[[1.0000, 0.0000],
          [0.4833, 0.5167]],

         [[1.0000, 0.0000],
          [0.5095, 0.4905]],

         [[1.0000, 0.0000],
          [0.4750, 0.5250]]]])
torch.Size([2, 3, 2, 2])


In [82]:
print(V)
print(V.shape)

tensor([[[[-0.3034,  0.6861, -0.3266,  0.7071],
          [ 1.1179,  0.8013,  0.3003,  0.8634]],

         [[-0.6100,  0.3836,  0.2797,  0.1567],
          [-0.9549,  0.8888,  0.3955, -0.6993]],

         [[ 1.7258, -1.8626, -1.0199, -0.5793],
          [-0.5443, -0.8104, -1.4201,  0.1152]]],


        [[[ 0.6855, -0.4818,  0.5789, -0.2515],
          [ 0.4977, -1.2107,  0.6248, -0.2825]],

         [[-1.2388, -0.0442,  0.0299, -0.1188],
          [ 1.0769, -0.2042, -0.2424, -1.2177]],

         [[-0.5631, -0.5505, -0.3765, -0.9081],
          [ 1.4550,  0.9858, -1.1073, -0.8797]]]])
torch.Size([2, 3, 2, 4])


In [107]:
#print(out)
print(out.shape)
out

torch.Size([2, 3, 2, 4])


tensor([[[[-0.3034,  0.6861, -0.3266,  0.7071],
          [ 0.4167,  0.7445, -0.0089,  0.7863]],

         [[-0.6100,  0.3836,  0.2797,  0.1567],
          [-0.7744,  0.6244,  0.3349, -0.2512]],

         [[ 1.7258, -1.8626, -1.0199, -0.5793],
          [ 0.5831, -1.3329, -1.2214, -0.2297]]],


        [[[ 0.6855, -0.4818,  0.5789, -0.2515],
          [ 0.5885, -0.8584,  0.6027, -0.2676]],

         [[-1.2388, -0.0442,  0.0299, -0.1188],
          [-0.1030, -0.1227, -0.1036, -0.6578]],

         [[-0.5631, -0.5505, -0.3765, -0.9081],
          [ 0.4964,  0.2561, -0.7602, -0.8932]]]])

In [ ]:
print(out.permute(0,2,1,3).shape)
out_permute = out.permute(0,2,1,3) # B, T, nh, hd
out_permute
#out_test
#out_test = out_test.view(B,T,nh*hd)

torch.Size([2, 2, 3, 4])


tensor([[[[-0.3034,  0.6861, -0.3266,  0.7071],
          [-0.6100,  0.3836,  0.2797,  0.1567],
          [ 1.7258, -1.8626, -1.0199, -0.5793]],

         [[ 0.4167,  0.7445, -0.0089,  0.7863],
          [-0.7744,  0.6244,  0.3349, -0.2512],
          [ 0.5831, -1.3329, -1.2214, -0.2297]]],


        [[[ 0.6855, -0.4818,  0.5789, -0.2515],
          [-1.2388, -0.0442,  0.0299, -0.1188],
          [-0.5631, -0.5505, -0.3765, -0.9081]],

         [[ 0.5885, -0.8584,  0.6027, -0.2676],
          [-0.1030, -0.1227, -0.1036, -0.6578],
          [ 0.4964,  0.2561, -0.7602, -0.8932]]]])

In [ ]:
out_contig = out_permute.contiguous()
out_contig.is_contiguous()

True

In [119]:
out_contig

tensor([[[[-0.3034,  0.6861, -0.3266,  0.7071],
          [-0.6100,  0.3836,  0.2797,  0.1567],
          [ 1.7258, -1.8626, -1.0199, -0.5793]],

         [[ 0.4167,  0.7445, -0.0089,  0.7863],
          [-0.7744,  0.6244,  0.3349, -0.2512],
          [ 0.5831, -1.3329, -1.2214, -0.2297]]],


        [[[ 0.6855, -0.4818,  0.5789, -0.2515],
          [-1.2388, -0.0442,  0.0299, -0.1188],
          [-0.5631, -0.5505, -0.3765, -0.9081]],

         [[ 0.5885, -0.8584,  0.6027, -0.2676],
          [-0.1030, -0.1227, -0.1036, -0.6578],
          [ 0.4964,  0.2561, -0.7602, -0.8932]]]])

In [122]:
print(out_contig.view(B,T,nh*hd).shape)
out_heads_concat = out_contig.view(B,T,nh*hd)
out_heads_concat

torch.Size([2, 2, 12])


tensor([[[-0.3034,  0.6861, -0.3266,  0.7071, -0.6100,  0.3836,  0.2797,
           0.1567,  1.7258, -1.8626, -1.0199, -0.5793],
         [ 0.4167,  0.7445, -0.0089,  0.7863, -0.7744,  0.6244,  0.3349,
          -0.2512,  0.5831, -1.3329, -1.2214, -0.2297]],

        [[ 0.6855, -0.4818,  0.5789, -0.2515, -1.2388, -0.0442,  0.0299,
          -0.1188, -0.5631, -0.5505, -0.3765, -0.9081],
         [ 0.5885, -0.8584,  0.6027, -0.2676, -0.1030, -0.1227, -0.1036,
          -0.6578,  0.4964,  0.2561, -0.7602, -0.8932]]])

In [124]:
ll = torch.nn.Linear(nh*hd,d,bias=True)
self_attention_out = ll(out_heads_concat)
print(self_attention_out.shape)
self_attention_out

torch.Size([2, 2, 3])


tensor([[[-0.0581,  0.3313, -1.2239],
         [-0.1983,  0.2745, -0.8868]],

        [[-0.2511,  0.2724, -0.1678],
         [-0.5068,  0.6644, -0.2228]]], grad_fn=<ViewBackward0>)

In [131]:
120 % 6

0